In [0]:
CREATE OR REPLACE TABLE crypto_lag_features AS
WITH latest_scrape AS (
    SELECT MAX(scrape_date) AS max_date
    FROM historical_crypto_trends
),
recent_symbols AS (
    SELECT DISTINCT symbol
    FROM historical_crypto_trends
    WHERE scrape_date = (SELECT max_date FROM latest_scrape)
),
ranked_data AS (
    SELECT
        h.symbol,
        h.name,
        h.market_cap,
        h.price,
        h.scrape_date,
        LAG(h.price, 1) OVER (PARTITION BY h.symbol ORDER BY h.scrape_date ASC) AS price_1,
        LAG(h.price, 2) OVER (PARTITION BY h.symbol ORDER BY h.scrape_date ASC) AS price_2,
        LAG(h.price, 3) OVER (PARTITION BY h.symbol ORDER BY h.scrape_date ASC) AS price_3,
        ((h.price - LAG(h.price, 1) OVER (PARTITION BY h.symbol ORDER BY h.scrape_date ASC)) 
            / LAG(h.price, 1) OVER (PARTITION BY h.symbol ORDER BY h.scrape_date ASC)) * 100 AS pct_change_1d,
        ((LAG(h.price, 1) OVER (PARTITION BY h.symbol ORDER BY h.scrape_date ASC) - 
          LAG(h.price, 2) OVER (PARTITION BY h.symbol ORDER BY h.scrape_date ASC)) 
            / LAG(h.price, 2) OVER (PARTITION BY h.symbol ORDER BY h.scrape_date ASC)) * 100 AS pct_change_2d,
        ((LAG(h.price, 2) OVER (PARTITION BY h.symbol ORDER BY h.scrape_date ASC) - 
          LAG(h.price, 3) OVER (PARTITION BY h.symbol ORDER BY h.scrape_date ASC)) 
            / LAG(h.price, 3) OVER (PARTITION BY h.symbol ORDER BY h.scrape_date ASC)) * 100 AS pct_change_3d
    FROM historical_crypto_trends h
    INNER JOIN recent_symbols r ON h.symbol = r.symbol
),
final AS (
    SELECT
        symbol,
        name,
        market_cap,
        price AS current_price,
        price_1,
        price_2,
        price_3,
        round(pct_change_1d,4) as pct_change_1d,
        round(pct_change_2d,4) as pct_change_2d,
        round(pct_change_3d,4) as pct_change_3d,
        STDDEV(pct_change_1d) OVER (PARTITION BY symbol) AS volatility_pct,
        scrape_date
    FROM ranked_data
)
SELECT *
FROM final
WHERE scrape_date = (SELECT max_date FROM latest_scrape)
  AND price_1 IS NOT NULL 
  AND price_2 IS NOT NULL 
  AND price_3 IS NOT NULL;

-- Successfully makes a table: crypto_lag_features







-- BACKUP
/*
WITH latest_scrape AS (
    -- Get the most recent scrape date
    SELECT MAX(scrape_date) AS max_date
    FROM historical_crypto_trends
),

recent_symbols AS (
    -- Only include cryptos that appeared in the most recent scrape
    SELECT DISTINCT symbol
    FROM historical_crypto_trends
    WHERE scrape_date = (SELECT max_date FROM latest_scrape)
),

ranked_data AS (
    -- Compute lagged prices for each symbol
    SELECT
        h.symbol,
        h.name,
        h.market_cap,
        h.price,
        h.scrape_date,
        LAG(h.price, 1) OVER (PARTITION BY h.symbol ORDER BY h.scrape_date ASC) AS price_1,
        LAG(h.price, 2) OVER (PARTITION BY h.symbol ORDER BY h.scrape_date ASC) AS price_2,
        LAG(h.price, 3) OVER (PARTITION BY h.symbol ORDER BY h.scrape_date ASC) AS price_3,
        
        -- Compute % change between each iteration
        ((h.price - LAG(h.price, 1) OVER (PARTITION BY h.symbol ORDER BY h.scrape_date ASC)) 
            / LAG(h.price, 1) OVER (PARTITION BY h.symbol ORDER BY h.scrape_date ASC)) * 100 AS pct_change_1d,

        ((LAG(h.price, 1) OVER (PARTITION BY h.symbol ORDER BY h.scrape_date ASC) - 
          LAG(h.price, 2) OVER (PARTITION BY h.symbol ORDER BY h.scrape_date ASC)) 
            / LAG(h.price, 2) OVER (PARTITION BY h.symbol ORDER BY h.scrape_date ASC)) * 100 AS pct_change_2d,

        ((LAG(h.price, 2) OVER (PARTITION BY h.symbol ORDER BY h.scrape_date ASC) - 
          LAG(h.price, 3) OVER (PARTITION BY h.symbol ORDER BY h.scrape_date ASC)) 
            / LAG(h.price, 3) OVER (PARTITION BY h.symbol ORDER BY h.scrape_date ASC)) * 100 AS pct_change_3d
    FROM historical_crypto_trends h
    INNER JOIN recent_symbols r ON h.symbol = r.symbol
),

final AS (
    -- Keep only the most recent scrape for display, include volatility + market share
    SELECT
        symbol,
        name,
        market_cap,
        price AS current_price,
        price_1,
        price_2,
        price_3,
        round(pct_change_1d,4) as pct_change_1d,
        round(pct_change_2d,4) as pct_change_2d,
        round(pct_change_3d,4) as pct_change_3d,
        STDDEV(pct_change_1d) OVER (PARTITION BY symbol) AS volatility_pct,
        scrape_date
    FROM ranked_data
)

SELECT *
FROM final
WHERE scrape_date = (SELECT max_date FROM latest_scrape)
  AND price_1 IS NOT NULL 
  AND price_2 IS NOT NULL 
  AND price_3 IS NOT NULL
ORDER BY name;
\*